In [1]:
import pandas as pd
import numpy as np
import zipfile
import json

In [99]:
zip_filename = "C:/Users/allen/allen/MADS/SIADS _697/data/cryptopunks.zip"
json_filename = "C:/Users/allen/allen/MADS/SIADS _697/data/0000.json"


df = pd.read_json(json_filename, orient='index')
#df = pd.read_json(json_filename, orient='index', compression='zip')
df = df.reset_index()
print(df)

                         index  \
0                           id   
1                    num_sales   
2             background_color   
3                    image_url   
4            image_preview_url   
5          image_thumbnail_url   
6           image_original_url   
7                animation_url   
8       animation_original_url   
9                         name   
10                 description   
11               external_link   
12              asset_contract   
13                   permalink   
14                  collection   
15                    decimals   
16              token_metadata   
17                       owner   
18                 sell_orders   
19                     creator   
20                      traits   
21                   last_sale   
22                     top_bid   
23                listing_date   
24                  is_presale   
25  transfer_fee_payment_token   
26                transfer_fee   
27                    token_id   

             

In [100]:
field = df[df['index'] == 'last_sale'][0]
print(field)
field_dic = field.to_list()[0]
print(field_dic.keys())
print(field_dic['total_price'])
print(field_dic['quantity'])
print(field_dic["event_timestamp"])
#print(field_dic['transaction'])

21    {'asset': {'decimals': None, 'token_id': '0'},...
Name: 0, dtype: object
dict_keys(['asset', 'asset_bundle', 'event_type', 'event_timestamp', 'auction_type', 'total_price', 'payment_token', 'transaction', 'created_date', 'quantity'])
0
1
2018-11-30T07:32:35


In [101]:
# open zipped dataset and create traits and sales dataframes by id

traits_df = None
full_traits_list = list()
full_sales_list = list()

with zipfile.ZipFile(zip_filename) as z:
    
    listOfiles = z.namelist()
    for fileName in listOfiles:
        #just the json
        if ".json" in fileName:

            with z.open(fileName) as f:
                #print(fileName)

                #creates traits dict list
                df = pd.read_json(f, orient='index')
                df = df.reset_index()
                traits_series = df[df['index'] == 'traits'][0]
                traits_list = traits_series.to_list()[0]
                #add id field to each trait_dict in list
                id_val = df[df['index'] == 'id'][0].to_list()[0]
                for traits_dict in traits_list:
                    traits_dict['id'] = id_val
                full_traits_list.extend(traits_list)
                
                #create costs list
                field_dic = df[df['index'] == 'last_sale'][0].to_list()[0]
        
                if field_dic is not None:
                
                    sales_dic = dict()
                    sales_dic['id'] = id_val
                    sales_dic['total_price'] = field_dic['total_price']
                    sales_dic['quantity'] = field_dic['quantity']
                    sales_dic['event_timestamp'] = field_dic['event_timestamp']
                    
                    sales_dic['eth_price'] = field_dic['payment_token']['eth_price']
                    sales_dic['usd_price'] = field_dic['payment_token']['usd_price']
                    
                    full_sales_list.append(sales_dic)
                

traits_df = pd.DataFrame(full_traits_list)
traits_df = traits_df.astype({"id": int, "trait_count": int})
print(traits_df.head(5))  

sales_df = pd.DataFrame(full_sales_list)
sales_df = sales_df.astype({"id": int, "total_price": float, "eth_price": float, "usd_price": float})
sales_df.head(5)   


  trait_type             value display_type max_value  trait_count order  \
0  accessory      3 attributes         None      None         2342  None   
1  accessory           Bandana         None      None          481  None   
2  accessory              Mole         None      None          644  None   
3       type            Female         None      None         3840  None   
4  accessory  Green Eye Shadow         None      None          271  None   

       id  
0  180242  
1  180242  
2  180242  
3  180242  
4  180242  


,id,total_price,quantity,event_timestamp,eth_price,usd_price
0,180242,9.000000e+16,1,2017-11-24T19:11:58,1.0,2607.28
1,178704,6.980000e+19,1,2021-08-02T00:45:14,1.0,2607.28
2,529015,1.650000e+19,1,2021-03-31T22:47:05,1.0,2607.28
3,178849,3.000000e+17,1,2018-12-16T04:06:39,1.0,2607.28
4,180205,3.300000e+19,1,2021-07-31T02:01:16,1.0,2607.28


In [94]:
traits_df.head(30)  

,trait_type,value,display_type,max_value,trait_count,order,id
0,accessory,3 attributes,None,None,2342,None,180242
1,accessory,Bandana,None,None,481,None,180242
2,accessory,Mole,None,None,644,None,180242
3,type,Female,None,None,3840,None,180242
4,accessory,Green Eye Shadow,None,None,271,None,180242
5,accessory,2 attributes,None,None,1909,None,178704
6,type,Male,None,None,6039,None,178704
7,accessory,Cowboy Hat,None,None,142,None,178704
8,accessory,Front Beard,None,None,273,None,178704
9,accessory,Knitted Cap,None,None,419,None,529015


In [95]:
#traits_df[traits_df['trait_type'] == "accessory"]['value'].unique()
traits_group_df = traits_df.groupby(['trait_type','value']).size().reset_index()

traits_group_df.rename({'0': 'count'}, axis=1, inplace=True)
traits_group_df.columns = ['trait_type', 'value', 'count']
traits_group_df = traits_group_df.sort_values(by=['count'], ascending=True)
traits_group_df.head(20)

,trait_type,value,count
8,accessory,7 attributes,1
0,accessory,0 attributes,8
95,type,Alien,9
7,accessory,6 attributes,11
96,type,Ape,24
10,accessory,Beanie,44
21,accessory,Choker,48
65,accessory,Pilot Helmet,54
86,accessory,Tiara,55
62,accessory,Orange Side,68


In [98]:

#sales_df['total_price'].plot(kind="bar", title="Total Price")
sales_df.describe()

,id,total_price,eth_price,usd_price
count,6601.000000,6.601000e+03,6601.0,6.601000e+03
mean,372043.411604,7.496175e+19,1.0,2.607280e+03
std,173311.128891,1.538824e+21,0.0,2.551326e-10
min,158697.000000,0.000000e+00,1.0,2.607280e+03
25%,178687.000000,3.900000e+18,1.0,2.607280e+03
50%,525920.000000,2.798000e+19,1.0,2.607280e+03
75%,528586.000000,8.175000e+19,1.0,2.607280e+03
max,531098.000000,1.244571e+23,1.0,2.607280e+03
